In [ ]:
import numpy as np
from scipy.io.wavfile import read
from scipy import signal, arange
import matplotlib.pyplot as plt
import matplotlib.axes as matax
import sys

In [ ]:
#samplingFrequency
sampFreq, sound = read('wav1.wav');


In [ ]:
#taking 5 seconds of sample from 15s - 20sec in file 
initTime = 15;
duration = 5;

In [ ]:
#start and end will hold the indeces in the sound sample array
start = sampFreq * initTime;
end = start + sampFreq * duration;

#s1 holds the spliced soundsample array
s1 = sound[start:end];

In [ ]:
#preparing timeArray for plotting. The plot will have x-axis in seconds (and not in indeces)
timeArray = np.arange(0, end-start, 1)
timeArray = timeArray / sampFreq

## This displayes the amplitude graph (with the points of interested noted in dots) and the spectrogram


In [ ]:
fig = plt.figure()
#fig, axes = plt.subplots(nrows=2, ncols=1)
ax1 = fig.add_axes( [0.12, 0.55, 0.75, 0.4] )
ax2 = fig.add_axes( [0.12, 0.1, 0.75, 0.4] )
ax3 = fig.add_axes( [0.88, 0.1, 0.02, 0.4] )
    
#--- Figure 1. ---#
ax1.plot(timeArray, s1)
ax1.set_ylabel('Amplitude')
ax1.set_xlim(0, duration)
ax1.margins(x=0)

#--- Figure 2. ---#
f, t, Sxx = signal.spectrogram(s1, sampFreq, nperseg=256)
im = ax2.pcolormesh(t, f/1000, np.log(Sxx), cmap='jet' )

#setting up the amplitude threshold
ampThreshold = np.std(s1) * 10
a = s1 > ampThreshold

#puts a mark at every point above the threshold
ax1.scatter(timeArray[a],s1[a])

ax2.set_ylabel('Frequency [kHz]')
ax2.set_ylim(0,20)
ax2.set_xlim(0, duration);
ax2.set_xlabel('Time (s)')
ax2.margins(x=0)
im.set_clim(-10, 0)

mappable = im
cb = plt.colorbar(mappable = mappable, cax = ax3)
cb.set_label('db/Hz')

plt.show();

#### Since 

In [ ]:

timeArray[a]
snaps = [];
snapIndex = [];

current = timeArray[a][0]
snaps.append(current)
snapIndex.append(current * sampFreq)

for time in timeArray[a]: 
    
    if (time > current + 0.01 ):
        snapIndex.append(time * sampFreq)
        snaps.append(time)
        current = time
        

    